# Intro to word embeddings

So far we have focused on bag-of-word approaches i.e representations of text as a vector of word frequencies. An alternative formalization of text consists in representing the words (or bi-grams, phrases, etc) themselves as vectors. A _word vector_ has no meaning per se, but it is informative of the _context_ in which the word is used. This vector representation can become very close to the semantic meaning of the word. Combined with simple vector operations, these representations can used to find synonyms, to test analogies, etc. Word vectors can also be used in any subsequent task (dictionary methods, classification, etc) as features instead of the simple word frequencies in the classical bag-of-words approach.

In this notebook we are going to construct word embeddings using neural networks. The spirit of the method is to use 'prediction as an excuse': either predict a target word conditional on its surrounding words (_continuous-bag-of-words_) or predict surrounding words conditional on the target (_skip-gram_). What we care about is not the final output but the _hidden layer_ projection from a two-layers neural network designed to solve that prediction problem (see skip-gram diagram below from Mikolov et al. 2013). 

<img src='img/wordembeddings_diagram.png' />

If we choose the hidden layer to have $K$ hidden neurons then each target word is represented by a $K$-dimensional vector of hidden outputm which we call _embedding_. In practice $K$ should be between 100 and 300, but that really depends on the vocabulary size. For the purpose of learning we are going to apply the famous skip-gram Google's Word2Vec approach (Mikolov et al. 2013) to a corpus that is typically _too small_ so that we reduce our word representations to vectors of 30 dimensions. Keep in mind that the typical required vocabulary size is at least half a million of unique tokens.

Last note on Word2Vec: it is very powerful! When trained on very large corpora (like all of English Wikipedia) it can perform very strong analogies such as finding that the vector corresponding the most to the output of the operation 'king' - 'man' + 'woman' is 'queen'. There exist alternative packages such as GloVe (Stanford NLP) or FastText.

## Data

We are going to embed the vocabulary from the corpus of Jane Austen's books we encountered on day 1. First, let's read the files into a single string:

In [1]:
import codecs

import os
DATA_DIR = 'data'

import glob
fnames = os.path.join(DATA_DIR, 'austen', '*.txt')
fnames = glob.glob(fnames)
raw = ''
for fname in fnames:
    with codecs.open(fname, "r", encoding='utf-8-sig', errors='ignore') as f:
        t = f.read()
        raw += t

In [2]:
raw[:1000]

'The Project Gutenberg EBook of Emma, by Jane Austen\r\n\r\nThis eBook is for the use of anyone anywhere at no cost and with\r\nalmost no restrictions whatsoever.  You may copy it, give it away or\r\nre-use it under the terms of the Project Gutenberg License included\r\nwith this eBook or online at www.gutenberg.org\r\n\r\n\r\nTitle: Emma\r\n\r\nAuthor: Jane Austen\r\n\r\nRelease Date: August, 1994  [Etext #158]\r\nPosting Date: January 21, 2010\r\nLast Updated: October 17, 2016\r\n\r\nLanguage: English\r\n\r\nCharacter set encoding: UTF-8\r\n\r\n*** START OF THIS PROJECT GUTENBERG EBOOK EMMA ***\r\n\r\n\r\n\r\n\r\nProduced by An Anonymous Volunteer\r\n\r\n\r\n\r\n\r\n\r\nEMMA\r\n\r\nBy Jane Austen\r\n\r\n\r\n\r\n\r\nVOLUME I\r\n\r\n\r\n\r\nCHAPTER I\r\n\r\n\r\nEmma Woodhouse, handsome, clever, and rich, with a comfortable home\r\nand happy disposition, seemed to unite some of the best blessings of\r\nexistence; and had lived nearly twenty-one years in the world with very\r\nlittle to 

## Quick pre-processing

In [3]:
text = raw [679:] # gets rid of meta information at the beginning

# A few modifications before sentence segmentation
text = text.replace('Mr.', 'Mr')
text = text.replace('Mrs.', 'Mrs')
text = text.replace('\n', ' ')
text = text.replace('\r', ' ')

# Sentence segmentation
import re
sent_boundary_pattern = r'[.?!]'
sentences = re.split(sent_boundary_pattern, text)

# Remove punctuation, special characters and upper cases
from string import punctuation
special = ['“', '”']
sentences = [''.join([ch for ch in sent if ch not in punctuation and ch not in special]) for sent in sentences]
sentences = [sent.lower() for sent in sentences]

# Remove white sace
sentences = [sent.strip() for sent in sentences]

# Tokenization within sentence
list_of_list = [sent.split() for sent in sentences]
list_of_list[:2]

[['emma',
  'woodhouse',
  'handsome',
  'clever',
  'and',
  'rich',
  'with',
  'a',
  'comfortable',
  'home',
  'and',
  'happy',
  'disposition',
  'seemed',
  'to',
  'unite',
  'some',
  'of',
  'the',
  'best',
  'blessings',
  'of',
  'existence',
  'and',
  'had',
  'lived',
  'nearly',
  'twentyone',
  'years',
  'in',
  'the',
  'world',
  'with',
  'very',
  'little',
  'to',
  'distress',
  'or',
  'vex',
  'her'],
 ['she',
  'was',
  'the',
  'youngest',
  'of',
  'the',
  'two',
  'daughters',
  'of',
  'a',
  'most',
  'affectionate',
  'indulgent',
  'father',
  'and',
  'had',
  'in',
  'consequence',
  'of',
  'her',
  'sister’s',
  'marriage',
  'been',
  'mistress',
  'of',
  'his',
  'house',
  'from',
  'a',
  'very',
  'early',
  'period']]

## Train a skip-gram model with Word2Vec 

First, you'll need to install [Gensim](https://pypi.org/project/gensim/). You can do so directly in the notebook using     ```!pip install```.

In [4]:
!pip install gensim

In [5]:
from gensim.models import Word2Vec

model = Word2Vec(min_count=2, size=30, sg=1)
model.build_vocab(list_of_list)  # prepare the model vocabulary
model.train(list_of_list, total_examples=model.corpus_count, epochs=model.iter)


C:\Users\Gregor von Richthof\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\Gregor von Richthof\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """


(2397717, 3378065)

## Asses model accuracy
### Size of vocabulary

In [6]:
 print(len(model.wv.vocab))

10244


### Latent vector representation

In [7]:
print(model.wv.word_vec('woman'))

[-0.22904672 -0.28888452  0.6314822   0.5529911  -0.5816087   0.37747845
  0.7699377   0.39877328 -1.1237882   0.07456231 -0.6187242   0.10058929
  0.3926051   0.81608933 -1.0285398   0.20173627 -0.22362886 -0.06954614
 -0.33557674  0.7103051   0.02760127  1.1307968  -0.2736073   0.3345016
  0.21023332  0.43452004 -0.11738215 -0.17811768  0.01778343  0.648622  ]


### Similarity between words

In [9]:
print(model.wv.similarity('woman', 'tree'))

0.5600547


C:\Users\Gregor von Richthof\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


### Most similar words 

In [10]:
print(model.wv.similar_by_word('woman'))

[('man', 0.9755523204803467), ('young', 0.8714357614517212), ('gentlemanlike', 0.8711866736412048), ('girl', 0.8699371814727783), ('wellmeaning', 0.8445643782615662), ('respectable', 0.8269561529159546), ('pleasing', 0.8267240524291992), ('lovely', 0.824920654296875), ('sensible', 0.8242414593696594), ('fellow', 0.8217689990997314)]


C:\Users\Gregor von Richthof\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [11]:
vector = model.wv.word_vec('woman') - model.wv.word_vec('man') + model.wv.word_vec('husband') #wife?
print(model.wv.similar_by_vector(vector))

[('husband', 0.9566090106964111), ('sister', 0.8778444528579712), ('aunt', 0.8750278949737549), ('father', 0.8676889538764954), ('mother', 0.8603571057319641), ('niece', 0.8498364090919495), ('encouraging', 0.847824215888977), ('friend', 0.8429862260818481), ('isabella', 0.8417069911956787), ('kindness', 0.8360919952392578)]


C:\Users\Gregor von Richthof\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


## Challenge

Try to improve the model by tuning its parameters:
- Increase the context window
- Construct continuous-bag-of-words representations
